<a href="https://akademie.datamics.com/kursliste/">![title](../screenshots/bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Check out our courses on <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em>

## Installation

#### Install Pytorch by using pip command from 
<a href="https://pytorch.org/get-started/locally/"> Pytorch official website</a>

In [ ]:
# !pip3 install torch torchvision torchaudio

### Create and run mlflow server

It connects the tracking URI, artifact and backend store to the non default locations

`mlflow server --default-artifact-root /Users/saumyagoyal/JupyterNotebook/Datamics/MLCon_Berlin/local_artifact_store  --backend-store-uri sqlite:////Users/saumyagoyal/JupyterNotebook/Datamics/MLCon_Berlin/sqlite_backend_store/backend_store.db --host 127.0.0.1 --port 5500 >> mlflow_server_log.txt`

<font color=#FF0000>**Note:** You can create mlflow server to run on a remote machine using the above command and connecting the backend (Example: AWS RDS instance) and artifact store(example: AWS S3 bucket) accordingly.</font>

## Sample Code

In [1]:
#### new Pytorch model training with MLFlow #### 

import torch
import math
import mlflow

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')


### Train the model ###

learning_rate = 0.1e-5
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print("Loss:", loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


#######################################################
################### MLflow code #######################
#######################################################

# setting the experiment details
experiment_name = "Experiment-9"
current_run_name = "new Backend store"
artifact_location = "/Users/saumyagoyal/JupyterNotebook/Datamics/MLCon_Berlin/local_artifact_store"
db_path = "/Users/saumyagoyal/JupyterNotebook/Datamics/MLCon_Berlin/sqlite_backend_store/backend_store.db"
db_uri = "sqlite:///"+db_path

# Set registry and tracking URI
mlflow.set_registry_uri(db_uri)
mlflow.set_tracking_uri(db_uri)

# adding tags for each run
tags = {"Demo": "True",
        "created-by": "dev team ID"}

# Creating new experiment  with artifact location only if the experiment doesn't exist
print(f'checking if the experiment {experiment_name} already exists or not')

if mlflow.get_experiment_by_name(experiment_name) == None :
    mlflow.create_experiment(experiment_name, artifact_location)
    mlflow.set_experiment(experiment_name)
    print(f"created a new experiment with experiment name {experiment_name}")
else:
    mlflow.set_experiment(experiment_name)
    print(f"Experiment {experiment_name} already exists, logging a new run with {current_run_name}")

# logging the current run
with mlflow.start_run(run_name = current_run_name):

    mlflow.set_tags(tags)

    # logging parameters - from the Pytorch code
    mlflow.log_param("learning rate",learning_rate)
    mlflow.log_param("loss", loss.item())

    # logging the model itself
    mlflow.pytorch.log_model(model, "Pytorch model")

    # Get storage location
    print("\n\nTracking URI: {}".format(mlflow.get_tracking_uri()))
    print("Artifact Location: {}".format(mlflow.get_artifact_uri()))
    print("--- Model logged successfully ---")


Loss: 332.771484375
Loss: 233.3954315185547
Loss: 164.6534881591797
Loss: 117.0536880493164
Loss: 84.06103515625
Loss: 61.17106628417969
Loss: 45.275299072265625
Loss: 34.22660827636719
Loss: 26.540191650390625
Loss: 21.188249588012695
Loss: 17.458749771118164
Loss: 14.857793807983398
Loss: 13.042466163635254
Loss: 11.774591445922852
Loss: 10.888429641723633
Loss: 10.268638610839844
Loss: 9.834870338439941
Loss: 9.531110763549805
Loss: 9.318270683288574
Loss: 9.169050216674805
checking if the experiment Experiment-9 already exists or not
created a new experiment with experiment name Experiment-9


Tracking URI: sqlite:////Users/saumyagoyal/JupyterNotebook/Datamics/MLCon_Berlin/sqlite_backend_store/backend_store.db
Artifact Location: /Users/saumyagoyal/JupyterNotebook/Datamics/MLCon_Berlin/local_artifact_store/0ffa690f491f438786ac15c15b758ddf/artifacts
--- Model logged successfully ---


### Check the logs in a new tracking server 
- Go to <a href='http://127.0.0.1:5500'> http://127.0.0.1:5500 </a>

![title](../screenshots/Pytorch_dashboard.png)

![title](../screenshots/Pytorch_model_registration.png)

<div class="alert alert-success">
    <h3> END </h3>
</div> 